# नामित इकाई मान्यता (NER)

यो नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) बाट लिइएको हो।

यस उदाहरणमा, हामी [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) डाटासेटमा आधारित NER मोडेल कसरी प्रशिक्षण गर्ने भनेर सिक्नेछौं। अघि बढ्नु अघि, कृपया [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) फाइललाई हालको डाइरेक्टरीमा डाउनलोड गर्नुहोस्।


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## डेटासेट तयार गर्दै

हामी डेटासेटलाई एक डेटा फ्रेममा पढेर सुरु गर्नेछौं। यदि तपाईं Pandas प्रयोग गर्ने बारेमा थप जान्न चाहनुहुन्छ भने, हाम्रो [Data Science for Beginners](http://aka.ms/datascience-beginners) को [डेटा प्रशोधनको पाठ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) हेर्नुहोस्।


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

अब हामीले शब्दावलीसँग पनि त्यही गर्न आवश्यक छ। सरलताका लागि, हामी शब्दको आवृत्तिलाई ध्यानमा नदिई शब्दावली बनाउनेछौं; वास्तविक जीवनमा तपाईंले Keras भेक्टराइजर प्रयोग गर्न चाहनुहुन्छ, र शब्दहरूको संख्या सीमित गर्न चाहनुहुन्छ।


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

हामीले प्रशिक्षणको लागि वाक्यहरूको डेटासेट बनाउन आवश्यक छ। आउनुहोस् मूल डेटासेटमा लूप गरौं र सबै व्यक्तिगत वाक्यहरूलाई `X` (शब्दहरूको सूची) र `Y` (टोकनहरूको सूची) मा अलग गरौं:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

सरलताका लागि, हामी सबै वाक्यहरूलाई अधिकतम लम्बाइसम्म 0 टोकनहरूद्वारा भरपर्दछौं। वास्तविक जीवनमा, हामीले अझ चतुर रणनीति प्रयोग गर्न चाहन सक्छौं, र केवल एक मिनिब्याचभित्र अनुक्रमहरूलाई भरपर्दछौं।


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## टोकन वर्गीकरण नेटवर्क परिभाषित गर्दै

हामी टोकन वर्गीकरणको लागि दुई-तह द्विदिशात्मक LSTM नेटवर्क प्रयोग गर्नेछौं। अन्तिम LSTM तहको प्रत्येक आउटपुटमा घना वर्गीकरणकर्ता लागू गर्नको लागि, हामी `TimeDistributed` निर्माण प्रयोग गर्नेछौं, जसले प्रत्येक चरणमा LSTM का प्रत्येक आउटपुटमा एउटै घना तह दोहोर्याउँछ:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

हाम्रो डेटासेटको लागि हामीले स्पष्ट रूपमा `maxlen` निर्दिष्ट गरिरहेका छौं - यदि हामी चाहन्छौं कि नेटवर्कले भिन्न लम्बाइका अनुक्रमहरूलाई ह्यान्डल गर्न सकोस्, भने नेटवर्क परिभाषा गर्दा हामीलाई अझ चतुर हुन आवश्यक छ।

अब हामी मोडेललाई प्रशिक्षण दिनेछौं। गति बढाउनका लागि, हामी केवल एक इपोकका लागि प्रशिक्षण दिनेछौं, तर तपाईं लामो समयसम्म प्रशिक्षण प्रयास गर्न सक्नुहुन्छ। साथै, तपाईंले डेटासेटको केही भागलाई प्रशिक्षण डेटासेटको रूपमा अलग गर्न चाहनुहुन्छ, ताकि मान्यताको शुद्धता अवलोकन गर्न सकियोस्।


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## परिणाम परीक्षण गर्दै

अब हाम्रा इकाई पहिचान मोडेलले एउटा नमूना वाक्यमा कसरी काम गर्छ हेर्न प्रयास गरौं:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## मुख्य कुरा

सामान्य LSTM मोडेलले पनि NER मा राम्रो नतिजा देखाउँछ। तर, अझ राम्रो नतिजा प्राप्त गर्नका लागि, तपाईंले BERT जस्ता ठूला प्रि-ट्रेन गरिएको भाषा मोडेलहरू प्रयोग गर्न चाहनुहुन्छ। Huggingface Transformers लाइब्रेरी प्रयोग गरेर NER को लागि BERT प्रशिक्षण गर्ने प्रक्रिया [यहाँ](https://huggingface.co/course/chapter7/2?fw=pt) वर्णन गरिएको छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धि हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
